In [ ]:
from pythtb import * # import TB model class
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import ase
import ase.io
import ase.neighborlist
import ase.visualize.plot

#sns.set_palette(sns.color_palette())
sns_colors = sns.color_palette(n_colors=2)

In [ ]:
M_param = 3
U_param = 16

In [ ]:
file_name = "../xyz_files/clars_goblet.xyz"
geom = ase.io.read(file_name)
lat = [
    [1.0,0.0],
    [0.0,1.0]
]
geom.center(about=(0, -1, 0))
orb = []

for at in geom:
    orb.append(at.position[:2])
    
x_max, y_max = np.max(orb, axis=0)
x_min, y_min = np.min(orb, axis=0)

max_coord_x = x_max + 1
min_coord_x = x_min - 1
max_coord_y = y_max + 1
min_coord_y = y_min - 1

In [ ]:
p_results_file = "../calculations/clars_goblet/clars_goblet_tb.txt"
p_f = open(p_results_file, "r")

p_num_lines = sum(1 for line in open(p_results_file, "r"))
p_evals = np.zeros(p_num_lines, dtype=float)
p_counter = 0

for p_line in p_f:
    p_spin = p_line[:1]
    p_line = p_line[2:]

    p_splitted_line = p_line.split(' ')

    p_evals[p_counter] = np.float64(p_splitted_line[0])
    p_counter = p_counter + 1

In [ ]:
results_file = "../calculations/clars_goblet/clars_goblet_tb.txt"
f = open(results_file, "r")

num_lines = sum(1 for line in open(results_file, "r"))
print("Number of lines in file: ", num_lines)
num_values_evecs = int(num_lines / 2)

In [ ]:
evals = np.zeros(num_lines, dtype=float)
occupancy = np.zeros(num_lines, dtype=float)
evecs_shape = (num_lines, num_values_evecs)
evecs = np.zeros(evecs_shape, dtype=complex)

total_spin_up = np.zeros(int(num_lines/2), dtype=float)
total_spin_down = np.zeros(int(num_lines/2), dtype=float)
up_spins = 0
down_spins = 0

evals_up = np.full(int(num_lines/2), -100, dtype=float)
evals_down = np.full(int(num_lines/2), -100, dtype=float)
counter_up = 0
counter_down = 0
    
counter = 0
word_count = 0

total_energy = 0

In [ ]:
for line in f:
    spin = int(line[:1])
    line = line[2:]

    splitted_line = line.split(' ')
    
    evals[counter] = np.float64(splitted_line[0])
    occupancy[counter] = int(splitted_line[1])
    
    # print("eval: ", evals[counter])
    # print("occ: ", occupancy[counter])
    
    splitted_line.pop(0)
    splitted_line.pop(0)
    
    # print(splitted_line)
    if(spin == 1):
        evals_up[counter_up] = evals[counter]
        counter_up = counter_up + 1
    elif(spin == 0):
        evals_down[counter_down] = evals[counter]
        counter_down = counter_down + 1
    else:
        print("Error.")
    
    print("Counter up: ", counter_up)
    print("Counter down: ", counter_down)
    
    x = ''
    y = ''
    y_turn = False
    
    print(counter)
    # print(word_count)
    # print(*splitted_line)
    # print("len of splitted line:", len(splitted_line))
    for j in range(int(num_lines/2)):
        for i in splitted_line[j]:
            if(i == '(' or i == ' '):
                continue               
            if(i != ',' and y_turn == False):
                x += i
            elif(i == ','):
                y_turn = True
                continue
            elif(i != ')'):
                y += i
            else:
                evecs[counter, word_count] = complex(float(x), float(y))
                x = ''
                y = ''
                word_count = word_count + 1
                y_turn = False
        
    
        
    if occupancy[counter] == 0:
        word_count = 0
        counter = counter + 1
        continue
    elif occupancy[counter] == 1:
        total_energy += evals[counter]
        if(spin == 1):
            up_spins = up_spins + 1
            #rint("evecs[counter] = ", evecs[counter])
            #print("np.square(evecs[counter]) = ", np.square(evecs[counter]))
            #print("np.absolute(evecs[counter]) = ", np.absolute(evecs[counter]))
            total_spin_up = np.add(total_spin_up, np.square(evecs[counter])) # replace absolute with square?
        elif(spin == 0):
            down_spins = down_spins + 1
            total_spin_down = np.add(total_spin_down, np.square(evecs[counter])) # replace absolute with square?
    else:
        print("Error.")

    word_count = 0
    counter = counter + 1
    if counter == num_lines:
        break

print("Total Energy: ", total_energy)
        
print("total up spins: " + str(up_spins))
print("total down spins: " + str(down_spins))
print(total_spin_up)
print(total_spin_down)
spin_difference = ((total_spin_up - total_spin_down))
print(np.shape(spin_difference))
print(spin_difference)

print("evals spin up: " + str(evals_up))
print("evals spin down: " + str(evals_down))

In [ ]:
ed = num_lines//2
print(num_lines, ed)
homo = ed-1

In [ ]:
plt.plot(evals[ed-4:ed+4], 'o')
plt.show()

plt.plot(evals[ed-12:ed+12], 'o')
plt.show()

plt.plot(evals[:], 'o')
plt.show()

for ev in evals[0:38]:
    print(ev)

In [ ]:
def visualize_backbone(ax, atoms):
    i_arr, j_arr = ase.neighborlist.neighbor_list('ij', atoms, 1.8)
    for i, j in zip(i_arr, j_arr):
        if i < j:
            p1 = atoms.positions[i]
            p2 = atoms.positions[j]
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'k-', linewidth=2.0, solid_capstyle='round')
            
def visualize_evec(ax, atoms, evec):
    for at, e in zip(atoms, evec):
        p = at.position
        mod = np.abs(e)
        phase = np.angle(e)/np.pi
        #col = (1.0-phase, 0.0, phase)
        col = sns_colors[int(phase)]
        circ = plt.Circle(p[:2], np.cbrt(mod), color=col, zorder=10)
        ax.add_artist(circ)

In [ ]:
fig=plt.figure(figsize=(8,8))
ax = plt.gca()
ax.set_aspect('equal')
visualize_backbone(ax, geom)
visualize_evec(ax, geom, spin_difference)
plt.axis('off')

plt.xlim([min_coord_x, max_coord_x])
plt.ylim([min_coord_y, max_coord_y])

path = "../figures/figures_clars_goblet/"
fig_name = path + "clars_goblet_tb.pdf".format(M_param, U_param)
#plt.savefig(fig_name, dpi=1000, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(6, 12))
plt.subplot(121)
ax = plt.gca()

past_ev = evals_down[0] - 1

counter = 0
for ev in evals_down:
    col = sns_colors[0]
    if counter < down_spins:
        col = sns_colors[1]
        print("Down spin ev: " + str(ev))
    counter += 1
    if(ev > past_ev + 0.08):
        plt.plot([0.55, 0.9], [ev, ev], '-', color=col, lw=3.0, solid_capstyle='round')
    else:
        plt.plot([0.1, 0.45], [ev, ev], '-', color=col, lw=3.0, solid_capstyle='round')
    past_ev = ev


ax.get_xaxis().set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)

upper_y_lim = evals[down_spins+up_spins] + 0.5 * abs(evals[0] - evals[down_spins+up_spins])
print()
plt.xlim(0.0, 1.0)
plt.ylim(evals[0]-1, evals[-1]+1)
plt.ylabel("Energy (eV)")

###

plt.subplot(122)
ax = plt.gca()

past_ev = evals_up[0] - 1
counter2 = 0
for ev in evals_up:
    col = sns_colors[0]
    if counter2 < up_spins:
        col = sns_colors[1]
        print("Up spin ev: " + str(ev))
    counter2 += 1
    if(ev > past_ev + 0.08):
        plt.plot([0.1, 0.45], [ev, ev], '-', color=col, lw=3.0, solid_capstyle='round')
    else:
        plt.plot([0.55, 0.9], [ev, ev], '-', color=col, lw=3.0, solid_capstyle='round')
    past_ev = ev
    
ax.get_xaxis().set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
plt.xlim(0.0, 1.0)
plt.ylim(evals[0]-1, evals[-1]+1)
#plt.ylabel("Energy (eV)")

#plt.savefig("../figures_phenalenyl/spin_eigenvalues.png", dpi=300, bbox_inches='tight')

path = "../figures/figures_clars_goblet/"
fig_name = path + "clars_goblet_energy_values_tb.pdf".format(M_param, U_param)
#plt.savefig(fig_name, dpi=1000, bbox_inches='tight')